In [1]:
# Basics of Langchain 

config = '.env.esri'
from dotenv import load_dotenv
load_dotenv(config)
import os
from langchain_openai import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate


system_prompt_template = """You are chatting with an AI assistant. The assistant is here to help you with any questions you have.

- Assume every request is about the location: {location}

Context:
{context}

History: 
{chat_history}

User Request: {request}

"""

llm = AzureChatOpenAI(
    temperature=0.9,
    model_name=os.environ["OPENAI_MODEL_NAME"], #'text-davinci-003'
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    deployment_name=os.environ["OPENAI_DEPLOYMENT_NAME"]
)

memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True)
# memory.load_memory_variables(history)


prompt = ChatPromptTemplate.from_template(system_prompt_template)

chat_agent = (
    {
        "request": RunnablePassthrough(),
        "context": RunnablePassthrough(),
        "chat_history": RunnablePassthrough(),
        "location": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [2]:
chat_agent.invoke({
    "request": "What city am I in?", 
    "context": "You are an elegant story teller who speaks in iambic pentameter",
    "location": "Seattle, WA",
    "chat_history": []
})

'You, my dear, are in the enchanting city of Seattle,\nWhere the emerald waters meet the sky so gentle.\nA place of beauty and charm, where tales unfold,\nIn iambic pentameter, your stories will be told.'

In [5]:
# Now let's use some tools to add some functionality to the agent

from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder
from tools.geocode import geocodeLocation
from langchain.tools import StructuredTool

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt_template),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{request}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

tools = [
    StructuredTool.from_function(geocodeLocation)
]
agent = create_openai_functions_agent(llm, tools, prompt)

data_agent = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    callbacks=[]
)


In [13]:
data_agent.invoke({
    "request": "where is city hall?", 
    "context": "",
    "location": "Seattle, WA",
    "chat_history": []
})



> Entering new AgentExecutor chain...

Invoking: `geocodeLocation` with `{'address': 'Seattle City Hall, Seattle, WA'}`


(-122.33048, 47.60351)The Seattle City Hall is located at the geographic coordinates:
- Longitude: -122.33048
- Latitude: 47.60351

> Finished chain.


{'request': 'where is city hall?',
 'context': '',
 'location': 'Seattle, WA',
 'chat_history': [],
 'output': 'The Seattle City Hall is located at the geographic coordinates:\n- Longitude: -122.33048\n- Latitude: 47.60351'}

In [14]:
data_agent.invoke({
    "request": "where is city hall?", 
    "context": "Provide your response in JSON",
    "location": "Seattle, WA",
    "chat_history": []
})



> Entering new AgentExecutor chain...

Invoking: `geocodeLocation` with `{'address': 'Seattle City Hall, Seattle, WA'}`


(-122.33048, 47.60351)```json
{
  "location": {
    "latitude": 47.60351,
    "longitude": -122.33048
  }
}
```

> Finished chain.


{'request': 'where is city hall?',
 'context': 'Provide your response in JSON',
 'location': 'Seattle, WA',
 'chat_history': [],
 'output': '```json\n{\n  "location": {\n    "latitude": 47.60351,\n    "longitude": -122.33048\n  }\n}\n```'}

In [11]:
data_agent.invoke({
    "request": "where is city hall?", 
    "context": "Provide javascript for displaying this location using an ArcGIS Map", #Provide your response in JSON
    "location": "Seattle, WA",
    "chat_history": []
})



> Entering new AgentExecutor chain...

Invoking: `geocodeLocation` with `{'address': 'City Hall, Seattle, WA'}`


(-122.3396, 47.46688)The geographic coordinates for City Hall in Seattle, WA are:
- Longitude: -122.3396
- Latitude: 47.46688

To display this location on an ArcGIS Map using Javascript, you can use the following code:

```javascript
require([
  "esri/Map",
  "esri/views/MapView",
  "esri/geometry/Point",
  "esri/symbols/SimpleMarkerSymbol",
  "esri/Graphic",
  "dojo/domReady!"
], function(Map, MapView, Point, SimpleMarkerSymbol, Graphic) {

  var map = new Map({
    basemap: "streets-navigation-vector"
  });

  var view = new MapView({
    container: "viewDiv",
    map: map,
    center: [-122.3396, 47.46688],
    zoom: 15
  });

  var point = new Point({
    longitude: -122.3396,
    latitude: 47.46688
  });

  var markerSymbol = new SimpleMarkerSymbol({
    color: [226, 119, 40],
    outline: {
      color: [255, 255, 255],
      width: 1
    }
  });

  var pointGraphic

{'request': 'where is city hall?',
 'context': 'Provide javascript for displaying this location using an ArcGIS Map',
 'location': 'Seattle, WA',
 'chat_history': [],
 'output': 'The geographic coordinates for City Hall in Seattle, WA are:\n- Longitude: -122.3396\n- Latitude: 47.46688\n\nTo display this location on an ArcGIS Map using Javascript, you can use the following code:\n\n```javascript\nrequire([\n  "esri/Map",\n  "esri/views/MapView",\n  "esri/geometry/Point",\n  "esri/symbols/SimpleMarkerSymbol",\n  "esri/Graphic",\n  "dojo/domReady!"\n], function(Map, MapView, Point, SimpleMarkerSymbol, Graphic) {\n\n  var map = new Map({\n    basemap: "streets-navigation-vector"\n  });\n\n  var view = new MapView({\n    container: "viewDiv",\n    map: map,\n    center: [-122.3396, 47.46688],\n    zoom: 15\n  });\n\n  var point = new Point({\n    longitude: -122.3396,\n    latitude: 47.46688\n  });\n\n  var markerSymbol = new SimpleMarkerSymbol({\n    color: [226, 119, 40],\n    outline: {\